In [ ]:
from dotenv import load_dotenv  
import os  
# Configure logger
import logging
import sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool
from azure.ai.agents.models import ListSortOrder
from azure.ai.agents.models import McpTool  # Correct import path for McpTool

# Load the .env file  
load_dotenv(override=True) 

# Now you can use the environment variables, for example:  
project_endpoint = os.environ["PROJECT_ENDPOINT"]
model=os.environ["MODEL_DEPLOYMENT_NAME"]

print(project_endpoint + " " + model)

In [ ]:
# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]


# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

In [ ]:

agent_name = os.environ["AZURE_MCP_AGENT_NAME"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    print(f"Agent '{agent_name}' not found in the project.")
else:
    logger.info(f"Agent '{agent_name}' found in the project.")
    logger.info(f"Agent ID: {agent.id}")

In [ ]:
mcp_server_url = "https://learn.microsoft.com/api/mcp"
mcp_server_label = "mslearn"

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
    exclude_environment_credential=True,
        exclude_managed_identity_credential=True
)

In [ ]:
if not agent:
    logger.info(f"Agent with name '{agent_name}' not found, creating a new agent.")

    # Initialize agent MCP tool
    mcp_tool = McpTool(
        server_label=mcp_server_label,
        server_url=mcp_server_url,
    )

    # Create a new agent with the mcp tool definitions
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="my-mcp-agent",
        instructions="""
        You have access to an MCP server called `microsoft.docs.mcp` - this tool allows you to 
        search through Microsoft's latest official documentation. Use the available MCP tools 
        to answer questions and perform tasks.""",
        tools=mcp_tool.definitions,
)
    logger.info(f"Created agent, ID: {agent.id}")
else:
    logger.info(f"Found agent by name '{agent_name}', ID={agent.id}")



In [ ]:
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

In [ ]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
    Give me the Azure CLI commands to create an Azure Container App iwth a managed idenitty
    """
)

print(f"Created message, ID: {message.id}")

In [ ]:
# Update mcp tool headers
mcp_tool.update_headers("SuperSecret", "123456")

In [ ]:
# Set approval mode
mcp_tool.set_approval_mode("never")

In [ ]:
# Create and process agent run in thread with MCP tools
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
print(f"Created run, ID: {run.id}")



In [ ]:
if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")